# lib

In [1]:
# primitive
import sys
import os
import pandas as pd

# handmade libs
src = '../../src'
if src not in sys.path: sys.path.append(src)
from const import *

# funcs

In [2]:
# For livedoor news corpus dataset
def get_files(dir_path):
    ngs = ['LICENSE.txt']
    files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f)) and f not in ngs]
    return files

def get_lines(file_path):
    with open(file_path, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    return [l for l in lines if len(l) > 0]

# proc

In [3]:
# dataset
p = '/data/livedoor/text/'
medium = [d for d in os.listdir(p) if os.path.isdir(os.path.join(p, d))]

dataset = []

for media in medium:
    dir_path = os.path.join(p, media)
    files = get_files(dir_path)
    
    for file in files:
        
        file_path = os.path.join(dir_path, file)
        lines = get_lines(file_path)
        
        url, timestamp, title, text = lines[0], lines[1], lines[2], ''.join(lines[3:])
        dataset.append((media, url, timestamp, title, text))

dataset = pd.DataFrame(dataset, columns=['media', 'url', 'timestamp', 'title', 'text'])
dataset = dataset[dataset.index!=6031].reset_index(drop=True)

# for prediction
dataset = dataset[['media', 'text']]

# label を数値化
itos = {i: s for i, s in enumerate(set(dataset['media']))}
stoi = {s: i for i, s in itos.items()}
dataset.loc[:, 'media'] = dataset['media'].map(stoi)

# # tmp
# dataset = dataset.sample(1234).reset_index(drop=True)

# 概観
print(dataset.shape)
display(dataset.head())
display(pd.DataFrame(dataset.media.value_counts()).sort_index().T.style.background_gradient('Blues', axis=1))

# output
f = os.path.join(ROOT, 'data', 'livedoor&text=text.csv')
if not os.path.isfile(f):
    dataset.to_csv(f, index=False)
    print('file created')
else:
    print('file exists')

(7366, 2)


,media,text
0,4,映画『鍵泥棒のメソッド』が、16日より開催中の第15回上海国際映画祭のコンペティション部門に...
1,4,監督・蜷川実花×主演・沢尻エリカで贈る『ヘルタースケルター』。本作のテーマソングが、浜崎あゆ...
2,4,「スーパー・ヒーローが強いのはもちろんだが、実際にはどれほどの実力なのだろうか？」という疑問...
3,4,自殺か？狂言か？──パニックとなった群衆が路上で見守る中、高層ホテルから飛び降りようとする一...
4,4,元・お嬢様の勅使河原 栄華（てしがわら えいか）は、父の事業の失敗で一転して貧乏人になるが、...


,0,1,2,3,4,5,6,7,8
media,899,770,870,870,870,511,864,842,870


file created
